In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib widget
matplotlib.rc('font', size=18)
import uncertainties as unc
import scipy.special as ss

from tqdm.notebook import tqdm

import sys
sys.path.append('../')
import general_purpose.utilities as ut
import general_purpose.uplotlib as uplt

from importlib import reload

import rea.reconstruct as rec


In [ ]:
# folder = './__test__/n--k__10--nens__10--T__5'
# folder = './__test__/i1--k__10--nens__10--T__5'
# folder = './__test__/i10--k__10--nens__10--T__5'
# folder = './__test__/i0m0--k__10--nens__1000--T__5'

# folder = './__test__/0--k__0--nens__999--T__10'
# folder = './__test__/rv--k__4--nens__999--T__10'
# folder = './__test__/rv--k__4--nens__20--T__10'
# folder = './__test__/0--k__4--nens__100--T__10'
# folder = './__test__/c--k__0--nens__999--T__10'
# folder = './__test__/c--k__0--nens__100--T__10'
# folder = './__test__/f1--k__4--nens__999--T__10'
# folder = './__test__/c0--k__0--nens__999--T__10'
# folder = './__test__/av_f0--k__4--nens__1000--T__50/'
# folder = './__test__/av_c0--k__0--nens__10000--T__1000'

# folder = '../veros/__test__/r1_minimal'
# folder = '../veros/__test__/r2_minimal'
# folder = '../veros/__test__/r3_minimal/'
# folder = '../veros/__test__/r01--k__16--nens__50--T__5_minimal'
# folder = '../veros/__test__/r2-relax_minimal'

folder = '../veros/__test__/rf1_minimal'

# folder = '../veros/__test__/t01--k__16--nens__50--T__10_minimal'
# folder = '../veros/__test__/t02--k__16--nens__5--T__15_minimal'
rec_dict = ut.json2dict(f'{folder}/reconstructed.json')

In [ ]:
reload(rec)

In [ ]:
rec_dict = rec.reconstruct(f'{folder}/i0015', write=True)
# rec_dict = rec.reconstruct(f'{folder}/i0011')

In [ ]:
rec_dict['folders']

In [ ]:
rec_trajs = {}

for rname, r in tqdm(rec_dict['members'].items()):
    traj = []
    for subfolder,e in zip(rec_dict['folders'], r['ancestry']):
        traj.append(np.load(f'{folder}/{subfolder}/{e}-traj.npy'))
    traj = np.concatenate(traj)

    rec_trajs[rname] = traj

# observables = {r: traj[-1,1] for r,traj in rec_trajs.items()}
observables = {r: traj[-1,1] - traj[0,1] for r,traj in rec_trajs.items()}
# observables = {r: np.mean(traj[-5:,1]) for r,traj in rec_trajs.items()}

In [ ]:
observables = {r: np.load(f'{folder}/i0019/e{i+1:03d}-traj.npy')[-1,1] for i,r in enumerate(observables)}

In [ ]:
## get the killed trajectories

nens = len(rec_dict['members'])
enss = set([f'e{i+1:0{len(str(nens))}d}' for i in range(nens)])

killed_trajs = []
for i,f in tqdm(enumerate(rec_dict['folders'])):
    kens = enss - set([r['ancestry'][i] for r in rec_dict['members'].values()])
    killed_trajs.append({ke: np.load(f'{folder}/{f}/{ke}-traj.npy') for ke in kens})

## Plot trajectories

In [ ]:
start=110
for traj in rec_trajs.values():
    plt.plot(traj[start:,0]-traj[0,0], -traj[start:,1], color='blue', alpha=0.2)
    
plt.axvline(traj[start,0]-traj[0,0], color='blue', linestyle='dashed')

In [ ]:
plt.close(1)
fig, ax = plt.subplots(num=1, figsize=(9,6))

for traj in rec_trajs.values():
    plt.plot(traj[:,0]-traj[0,0], -traj[:,1], color='gray', alpha=0.2)
    
# for seg in killed_trajs:
#     for traj in seg.values():
#         plt.plot(traj[:,0], -traj[:,1], color='red', alpha=0.2)
    
# plt.axhline(0, color='black', linestyle='dashed')

plt.xlabel('t [years]')
plt.ylabel('AMOC strenght [Sv]')
# plt.title('N = 1000, k=0')

fig.tight_layout()

In [ ]:
(1000-680)/20

In [ ]:
traj.shape

In [ ]:
for e in rec_dict['members'].items():
    print(e)

In [ ]:
# check the scores

In [ ]:
first_info = ut.json2dict(f"{folder}/{rec_dict['folders'][0]}/info.json")
last_info = ut.json2dict(f"{folder}/{rec_dict['last_folder']}/info.json")
for rname, e in rec_dict['members'].items():
    ename = rname.replace('r','e')
    
    cum_score = last_info['members'][ename]['cum_score_i'] - first_info['members'][e['ancestry'][0]]['cum_score_i']
    cum_log_escore = last_info['members'][ename]['cum_log_escore_i'] - first_info['members'][e['ancestry'][0]]['cum_log_escore_i']
    print(rname, cum_score - e['cum_score'], cum_log_escore - e['cum_log_escore'])

In [ ]:
plt.close('all')

In [ ]:
np.min(list(observables.values()))

In [ ]:
rec_dict['independent_parents']

## Compute expectations

In [ ]:
def f(x, a):
    return float(x > a)

def expectation(func, **kwargs):
    _f = np.array([func(x, **kwargs) for x in observables.values()])
    # print(np.mean(_f))
    _w = np.array([r['weight'] for r in rec_dict['members'].values()])
    # print(np.sum(_w))
    # _w = np.exp(-np.array([r['cum_log_escore'] for r in rec_dict['members'].values()]))
    # _n = np.exp(rec_dict['cum_log_norm_factor'])

    mean = np.mean(_f*_w)
    std = np.sqrt(np.mean(_f*_w**2)/len(_w))

    return unc.ufloat(mean,std)

In [ ]:
min_obs = np.min(list(observables.values()))
max_obs = np.max(list(observables.values()))

print(min_obs, max_obs)

In [ ]:
expectation(f, a=-8.1)

In [ ]:
max_obs = -7.5
min_obs = -8.3

In [ ]:
a = np.linspace(min_obs*0.99, max_obs*1.01, 100)
probs = np.array([expectation(f, a=_a) for _a in a])

In [ ]:
traj.shape

In [ ]:
traj[-1,0]-traj[0,0] + 5

In [ ]:
plt.close(3)
fig, ax = plt.subplots(num=3, figsize=(9,6))

uplt.errorband(-a,probs, label=f'After {traj[-1,0]-traj[0,0] + 5:.0f} years')
plt.yscale('log')
# plt.ylim(1e-5,3)
plt.axhline(1, color='black', linestyle='dashed')

plt.xlabel('AMOC strenght [Sv]')
plt.ylabel('probability')

# plt.title(f'After {traj[-1,0]-traj[0,0] + 5:.0f} years')

fig.tight_layout()

In [ ]:
uplt.errorband(-a,probs, label=f'After {traj[-1,0]-traj[0,0] + 5:.0f} years')

In [ ]:
plt.xlabel('AMOC strenght variation [Sv]')
plt.legend()

## Theoretical probability of overcoming threshold

In [ ]:
lam = 1
sig = 1
def ou_var(t):
    return sig**2/(2*lam)*(1 - np.exp(-2*lam*t))

cum_gaus = lambda x: 0.5*(1 + ss.erf(x/np.sqrt(2)))

overcoming_prob = lambda x,v: cum_gaus(-x/np.sqrt(v))

In [ ]:
t = 0.1
v = ou_var(t)

In [ ]:
a = np.linspace(0,4,101)

e = np.array([expectation(f, a=_a) for _a in a])
e_th = overcoming_prob(a,v)

In [ ]:
plt.close(2)
fig,ax = plt.subplots(figsize=(9,6))

uplt.plot(a,e)
plt.plot(a, e_th)
# plt.plot(a,uplt.std_dev(e)/uplt.nominal_value(e))
# plt.plot(a, uplt.nominal_value(e)/e_th - 1)

fig.tight_layout()

In [ ]:
uplt.plot(a,e)
# plt.plot(a,uplt.std_dev(e)/uplt.nominal_value(e))